In [1]:
import numpy as np
import pandas as pd
from generate_daily_temp import daily_temp_interp, create_noise, daily_temp_era5
from rr_calculation import get_all_data_masked, get_pop_weighted_factors, get_paf_to_rr, get_array_from_mask, get_temp_array_from_mask
from read_files import get_annual_pop, read_climate_data, read_greg_and_TempZone, open_all_diseases_data

In [ ]:
### Define some variables and load important data
ssp =  'SSP2_CP' # 'SSP1_M', SSP2_CP, SSP3_H, SSP5_H
years = [2019] # range(2010,2101)
std_values = [1] # [1,5,10]
ccategories = ['C1'] # ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8']

gcm_diff, gcm_start, gcm_end, cc_path_mean = read_climate_data()
greg, era5_tz = read_greg_and_TempZone()
diseases, erf, min_dict, max_dict = open_all_diseases_data()
pop_ssp = get_annual_pop(ssp)

In [ ]:
### Main code ###
for std in std_values:
    # Generate the daily temperature data variability
    noise, noise_leap = create_noise(std)
    
    for ccategory in ccategories:
        rr_year = pd.DataFrame(index=range(1,27), columns=pd.MultiIndex.from_product([years, diseases]))            
        df_counts = pd.DataFrame(index=pd.MultiIndex.from_product([np.arange(1,27.,1), np.arange(-40,60.,0.1, dtype='float32').round(1)]))
        
        for year in years:
            print(f'{ccategory} {year} started')
            ### Generate annual daily temperature using dummy data (first line) or ERA5 reanalysis data (upon avaliability in folder)
            # daily_temp, num_days = daily_temp_interp(ccategory, year, cc_path_mean, gcm_diff, gcm_start, gcm_end, noise, noise_leap)
            daily_temp, num_days = daily_temp_era5(year, pop_ssp)
            # Mask the relevant datasets with a layer of population data
            greg_array, pop_array, meanTemp_array, dayTemp_array = get_all_data_masked(pop_ssp, year, num_days, greg, era5_tz, daily_temp)
            # Calculate the population weighted factor for each daily temperature (1st decimal) in a given region and temperature zone
            df = get_pop_weighted_factors(greg_array, pop_array, meanTemp_array, np.round(dayTemp_array,1))   
            # Add daily temperature values to df_counts
            df_counts[f'{year}'] = df.groupby(['greg','dailyTemp'])['pop'].sum()
            # Clip dailyTemp values according to the temperatures available in the ERF, get PAF, aggregate and calculate RR
            rr_year[year] = get_paf_to_rr(df, min_dict, max_dict, erf, diseases)
            print(f'{year} done') ## Aproxx 25s per year

        # Save the results and temperature statistics
        # rr_year.to_csv(f'X:\\user\\liprandicn\\Health Module\\output\\rr_m1_all-diseases_{ccategory}_2010-2100_{std}std.csv')
        rr_year.to_csv(f'X:\\user\\liprandicn\\Health Module\\output\\rr_m1_all-diseases_{year}_era5.csv')
        # df_counts.to_csv(f'X:\\user\\liprandicn\\Health Module\\output\\dailyTemp_{ccategory}_2010-2100_{std}std.csv')    
        df_counts.to_csv(f'X:\\user\\liprandicn\\Health Module\\output\\dailyTemp_{year}_era5.csv')   